# DAVID RODRIGUES ALBUQUERQUE - Tarefa Final - Snake
### Introdução ao Aprendizado de Máquina - 2023.1 - Ciência da Computação

Feito por:

## David Rodrigues Albuquerque - 120047390

---

In [1]:
%pip install pygame
import random
import numpy as np
import pandas as pd
import pygame
import sys
import time

Keyring is skipped due to an exception: 'keyring.backends'


Note: you may need to restart the kernel to use updated packages.
pygame 2.5.0 (SDL 2.28.0, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Snake:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.snake = [(width // 2, height // 2)]
        self.direction = 'up'
        self.apple = self.generate_apple()
        self.score = 0
        self.train_game_over = False

    def generate_apple(self):
        while True:
            apple = (random.randint(0, self.width - 1),
                     random.randint(0, self.height - 1))
            if apple not in self.snake:
                return apple

    def change_direction(self, direction):
        if direction == 'up' and self.direction != 'down':
            self.direction = 'up'
        elif direction == 'down' and self.direction != 'up':
            self.direction = 'down'
        elif direction == 'left' and self.direction != 'right':
            self.direction = 'left'
        elif direction == 'right' and self.direction != 'left':
            self.direction = 'right'

    def move(self):
        head = self.snake[0]
        if self.direction == 'up':
            new_head = (head[0], (head[1] - 1))
        elif self.direction == 'down':
            new_head = (head[0], (head[1] + 1))
        elif self.direction == 'left':
            new_head = ((head[0] - 1), head[1])
        elif self.direction == 'right':
            new_head = ((head[0] + 1), head[1])

        found_apple = False

        if (
            new_head[0] < 0 or new_head[0] >= self.width or
            new_head[1] < 0 or new_head[1] >= self.height or
            new_head in self.snake
        ):
            self.train_game_over = True
            return True, found_apple, self.get_state()

        self.snake.insert(0, new_head)
        if new_head == self.apple:
            self.score += 1
            self.apple = self.generate_apple()
            found_apple = True
        else:
            self.snake.pop()

        return False, found_apple, self.get_state()
    
    def input_action(self, action):
        # Aplicar a ação no jogo
        self.change_direction(action)
        died, _, _ = self.move()

        if died:
            return False
        
    def get_state(self):
        head_pos = self.snake[0]
        apple_pos = self.apple
        return head_pos[0], head_pos[1], apple_pos[0], apple_pos[1], self.direction
    
    def get_state_index(self, state):
        # Retorna os indices da posição da cobra e da maçã
        x_head, y_head, x_apple, y_apple, direction = state

        # Garantir que as posições estejam dentro dos limites da grade
        x_head = min(max(x_head, 0), 10 - 1)
        y_head = min(max(y_head, 0), 10 - 1)
        x_apple = min(max(x_apple, 0), 10 - 1)
        y_apple = min(max(y_apple, 0), 10 - 1)

        # Calcular o índice do estado na tabela Q
        state_index = x_head * 10 + y_head
        apple_index = x_apple * 10 + y_apple

        return state_index, apple_index

    def select_action(self, q_table, state, epsilon):
        state_index, apple_index = self.get_state_index(state)        

        if np.random.random() < epsilon:
            # Escolha uma ação aleatória
            action_index = np.random.randint(4)
        else:
            # Obtenha o índice da ação com o maior valor Q para o estado atual
            action_index = np.argmax(q_table[state_index, apple_index - 1])

        return action_index

    def check_collision(self):
        head = self.snake[0]
        if (
            head[0] < 0 or head[0] >= self.width or
            head[1] < 0 or head[1] >= self.height or
            head in self.snake[1:]
        ):
            return True
        return False

In [3]:
class Game:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.snake = Snake(width, height)
        self.game_over = False

        pygame.init()
        self.clock = pygame.time.Clock()
        self.window = pygame.display.set_mode((width * 30, height * 30))
        pygame.display.set_caption('Snake Game')

    def run(self):
        # Define a taxa de atualização do jogo (frames por segundo)
        self.clock.tick(15)
        pygame.display.flip()
        # self.handle_events()
        #self.update()
        self.render()
        
        if self.game_over:
            self.quit()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    self.snake.change_direction('up')
                elif event.key == pygame.K_DOWN:
                    self.snake.change_direction('down')
                elif event.key == pygame.K_LEFT:
                    self.snake.change_direction('left')
                elif event.key == pygame.K_RIGHT:
                    self.snake.change_direction('right')

    def update(self):
        if self.snake.move():
            self.game_over = True

    def render(self):
        self.window.fill((0, 0, 0))  # Preenche a janela com a cor preta
        for segment in self.snake.snake:
            # Desenha cada segmento da cobra
            pygame.draw.rect(self.window, (255, 255, 255),
                             (segment[0] * 20, segment[1] * 20, 20, 20))
        pygame.draw.rect(self.window, (255, 0, 0), (
            self.snake.apple[0] * 20, self.snake.apple[1] * 20, 20, 20))  # Desenha a maçã
        pygame.display.flip()  # Atualiza a janela

    def show_game_over_menu(self):
        font = pygame.font.SysFont(None, 36)
        text = font.render('Game Over!', True, (255, 0, 0))
        self.window.blit(text, (self.width * 10 - text.get_width() //
                         2, self.height * 10 - text.get_height() // 2))
        text = font.render('Press R to restart or Q to quit',
                           True, (255, 255, 255))
        self.window.blit(text, (self.width * 10 - text.get_width() //
                         2, self.height * 10 + text.get_height() // 2 + 10))
        pygame.display.flip()

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.quit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        self.reset()
                        return
                    elif event.key == pygame.K_q:
                        self.quit()

    def reset(self):
        self.snake = Snake(self.width, self.height)
        self.game_over = False

    def quit(self):
        pygame.display.flip()
        pygame.quit()
        sys.exit()


In [4]:
# Executando o jogo
# snake = Game(30, 30)  # Define a largura e altura do grid do jogo

# while True:
#     snake.run()
#     if not snake.game_over:
#         break


In [5]:
class Q_Table:
    def __init__(self, alpha, gamma, epsilon, snake: Snake):
        self.epsilon = epsilon # Taxa de exploracao
        self.alpha = alpha # Taxa de aprendizado
        self.gamma = gamma # Taxa de desconto
        self.min_epsilon = 0.001
        self.epsilon_decay_rate = 0.99992

        self.actions = ["up", "down", "left", "right"]
        self.snake = snake

        # Tamanho da grade do jogo
        N = 10
        M = 10

        # Número de ações possíveis
        K = 4

        # Inicializando a tabela Q com zeros
        self.q_table = np.zeros((N * M, N * M - 1, K))


    def step(self, action=None):
        if action == "None":
            action = random.choice(["left", "right", "up", "down"])
        else:
            action = ["left", "right", "up", "down"][action]

        if action == "left":
            self.snake.change_direction("left")
        elif action == "right":
            self.snake.change_direction("right")
        elif action == "up":
            self.snake.change_direction("up")
        elif action == "down":
            self.snake.change_direction("down")

        reward = 0
        state = self.snake.get_state()

        died, found_apple, next_state = self.snake.move()

        if died:
            # Recompensa por morrer = -10
            reward = -10
        elif found_apple:
            # Recompensa por achar maçã = 5
            reward = 10
        else:
            reward = -1
        
        state = self.update_q_table(state, action, reward, next_state)

    def update_q_table(self, state, action, reward, next_state):
        # Obtém os índices correspondentes ao estado e ação
        action_index = self.actions.index(action)
        state_index, apple_index = self.snake.get_state_index(state)
        next_state_index, next_apple_index = self.snake.get_state_index(next_state)

        self.epsilon = max(self.epsilon * self.epsilon_decay_rate, self.min_epsilon)

        # Obtém o valor atual da tabela Q para o estado e ação
        current_q_value = self.q_table[state_index, apple_index - 1, action_index]

        # Calcula o novo valor de Q com base na fórmula do algoritmo Q-Learning
        ## Equação de Bellman
        ### Q(s, a) += alpha * (r(s, a) + gamma * max Q(s', .) - Q(s, a))
        new_q_value = current_q_value + self.alpha * \
            (reward + self.gamma *
             np.max(self.q_table[next_state_index, next_apple_index - 1]) - current_q_value)
        
        # Atualiza o valor na tabela Q
        self.q_table[state_index, apple_index - 1, action_index] = new_q_value

# Execução do Treinamento Bruto

In [6]:
# Game Loop Training

alpha = 0.2 # Taxa de aprendizado
gamma = 0.9 # Taxa de desconto
epsilon = 1 # Taxa de exploração
num_episodes = 10000
width = 10
height = 10

q_table = Q_Table(alpha, gamma, epsilon, None)

# Loop principal para executar o jogo e treinar
for episode in range(num_episodes):
    # Crie uma instância do jogo Snake
    game = Game(width, height)
    snake = game.snake
    q_table.snake = snake

    # Inicialize o estado inicial
    state = snake.get_state()

    while not snake.train_game_over:
        # Selecione a ação com base na estratégia epsilon-greedy
        action = snake.select_action(q_table.q_table, state, q_table.epsilon)

        # Execute a ação no jogo
        snake.input_action(action)

        # Obtenha o próximo estado após a ação
        next_state = snake.get_state()

        # Atualize a tabela Q com base na ação, estado atual e próximo estado
        q_table.step(action)

        # Atualize o estado atual para o próximo estado
        state = next_state

    # Imprima o resultado do episódio
    print(f"Episode {episode+1}: Score = {snake.score} - Epsilon = {q_table.epsilon}")
    
print(q_table)



Episode 1: Score = 0 - Epsilon = 0.9989604990535976
Episode 2: Score = 0 - Epsilon = 0.9984012154165188
Episode 3: Score = 0 - Epsilon = 0.9980019188229187
Episode 4: Score = 1 - Epsilon = 0.9976825965301253
Episode 5: Score = 0 - Epsilon = 0.9971240283467326
Episode 6: Score = 0 - Epsilon = 0.9968847377242004
Episode 7: Score = 0 - Epsilon = 0.9965657728864611
Episode 8: Score = 1 - Epsilon = 0.9961672103524138
Episode 9: Score = 0 - Epsilon = 0.9959281493478298
Episode 10: Score = 0 - Epsilon = 0.9954501994350476
Episode 11: Score = 0 - Epsilon = 0.9951316935944776
Episode 12: Score = 0 - Epsilon = 0.9947337046003735
Episode 13: Score = 0 - Epsilon = 0.9944154280106385
Episode 14: Score = 0 - Epsilon = 0.9940177254769306
Episode 15: Score = 0 - Epsilon = 0.9936201819987852
Episode 16: Score = 0 - Epsilon = 0.9932227975125905
Episode 17: Score = 0 - Epsilon = 0.9928255719547596
Episode 18: Score = 0 - Epsilon = 0.9925079058942029
Episode 19: Score = 1 - Epsilon = 0.9921903414745878
Ep

# Execução do Treinamento Observando Jogar

In [7]:
# # Game Loop Training

# alpha = 0.01 # Taxa de aprendizado
# gamma = 0.9 # Taxa de desconto
# epsilon = 1 # Taxa de exploração
# num_episodes = 10000
# width = 10
# height = 10

# q_table = Q_Table(alpha, gamma, epsilon, None)

# # Loop principal para executar o jogo e treinar
# for episode in range(num_episodes):
#     # Crie uma instância do jogo Snake
#     game = Game(width, height)
#     snake = game.snake
#     q_table.snake = snake

#     # Inicialize o estado inicial
#     state = snake.get_state()

#     while not snake.train_game_over:
#         # Selecione a ação com base na estratégia epsilon-greedy
#         action = snake.select_action(q_table.q_table, state, q_table.epsilon)

#         # Execute a ação no jogo
#         snake.input_action(action)

#         # Obtenha o próximo estado após a ação
#         next_state = snake.get_state()

#         # Atualize a tabela Q com base na ação, estado atual e próximo estado
#         q_table.step(action)

#         game.run()

#         # Atualize o estado atual para o próximo estado
#         state = next_state
        

#     # Imprima o resultado do episódio
#     print(f"Episode {episode+1}: Score = {snake.score} - Epsilon = {q_table.epsilon}")

# print(q_table)
# game.quit()


# Objetivo


# Referências Bibliográficas
